# Exploiter le guide des recettes
Dans cet exercice, nous introduisons le Retrieval Augmented Generation (RAG), une approche qui permet à un agent basé sur un LLM de s’appuyer sur des données externes pour générer des réponses plus fiables, précises et à jour. Plutôt que de se reposer uniquement sur les connaissances internes du modèle, le LLM va d’abord rechercher des informations pertinentes dans une base de documents, puis utiliser ces informations comme contexte pour produire sa réponse.

Le RAG est particulièrement utile lorsque l’on travaille avec des documents spécifiques (documentation, articles, données métier) ou lorsque l’on souhaite éviter les hallucinations. Dans cet exercice, nous allons combiner DSPy pour l’orchestration du raisonnement et Qdrant comme base de données vectorielle pour la recherche sémantique.

## Définition des variables
Les variables sont lues depuis le fichier [.env](../../.env)

In [ ]:
from dotenv import dotenv_values
config = dotenv_values("../../.env")

llm_model = config.get('LLM_MODEL')
api_key = config.get('LLM_API_KEY')
# Uncomment for local api call
# api_base = config.get('LLM_API_URL')


qdrant_url = config.get('QDRANT_URL')
qdrant_api_key = config.get('QDRANT_API_KEY')

## Configuration du llm sur dspy

In [ ]:
import dspy
from dspy_qdrant import QdrantRM
from qdrant_client import QdrantClient
from langchain_huggingface import HuggingFaceEmbeddings

# Ton vectorizer (embedding) francophone
hf_embed = HuggingFaceEmbeddings(
    model_name="manu/bge-fr-en",  # modèle francophone + multilingue
    encode_kwargs={"normalize_embeddings": True}
)

def vectorizer(text_or_texts):
    if isinstance(text_or_texts, list):
        return hf_embed.embed_documents(text_or_texts)
    else:
        return hf_embed.embed_query(text_or_texts)


lm = dspy.LM(llm_model, api_key=api_key)
# Uncomment for local api call
#lm = dspy.LM(llm_model, api_base=api_base, track_usage=True, temperature=1.5, max_tokens=1024)

client = QdrantClient(qdrant_url, api_key=qdrant_api_key)
rm = QdrantRM(
    qdrant_collection_name="livre-recette", 
    qdrant_client=client,
    vector_name="recette",
    document_field="text",
    vectorizer=vectorizer
)


dspy.configure_cache(
    enable_disk_cache=False,
    enable_memory_cache=False,
)
dspy.configure(lm=lm, rm=rm)

## Créer le Retriever

Cette section définit la fonction `retriever` qui permet de rechercher les passages pertinents dans la base de données vectorielle. Elle utilise l'objet `dspy.Retrieve` pour effectuer une recherche sémantique basée sur la requête utilisateur et retourne un certain nombre de résultats (défini par le paramètre `k`).

In [ ]:


def retriever(query: str, k: int) -> list[str]:
    retriever = dspy.Retrieve(k=k)
    revelant_passages = retriever(query).passages
    print(f"Revelant passages: {revelant_passages}")
    return revelant_passages

## Enrichir la génération

Cette section définit l'assistant qui utilise le raisonnement en chaîne (Chain of Thought) pour enrichir la réponse finale avec le contexte récupéré. Elle combine les passages pertinents de la base de données avec la question utilisateur dans un format spécifique pour produire une réponse plus précise et fondée sur des sources externes.

In [ ]:

assistant = dspy.ChainOfThought("context, question -> response")

def rag(query: str, k: int) -> list[str]:
    revelant_passages = retriever(query, k)
    return assistant(context=revelant_passages, question=query)

## Exécuter le RAG

Cette section montre comment exécuter le système RAG complet. Elle appelle la fonction `rag` avec une requête utilisateur spécifique et un nombre de passages à récupérer (k=2). Le résultat est affiché à l'aide de la bibliothèque rich, permettant d'obtenir une sortie formatée et l'historique des appels du modèle pour analyse.

In [ ]:
from rich import print
results = rag(query="Comment cuisiner un nachos garnis ?", k=2)

print(results)
print(lm.history)
dspy.inspect_history()